In [4]:

print("Fichier importé !")

Fichier importé !


In [5]:
import re
from pyspark.sql import SparkSession

# 1. Initialiser la SparkSession (le point d'entrée)
spark = SparkSession.builder.appName("FakeNewsWordCount").getOrCreate()

# 2. Obtenir le 'SparkContext' (le cœur de l'API RDD)
sc = spark.sparkContext

# 3. Définir des "stop words" simples (mots à ignorer)
stop_words = set([
    "the", "a", "an", "is", "are", "was", "were", "in", "on", "at", "of", "to", "for",
    "and", "or", "but", "it", "he", "she", "they", "we", "i", "you", "that", "this", "would", "when",
    "with", "as", "by", "from", "said", "its", "has", "had", "not", "be", "will", "have", "like", "what"
])


# --- Modélisation des RDD (Mission 4) ---

# RDD 1 : baseRDD (charge le fichier texte)
# Chaque élément est une ligne (String)
baseRDD = sc.textFile("articles.tsv")


# RDD 2 : flatMappedRDD (Le "Mapper")
# C'est l'étape la plus complexe : Map + flatMap en une seule fois
# Elle transforme les lignes en paires clé-valeur de ( (Label, Mot), 1 )
def map_line_to_words(line):
    parts = line.split('\t')
    if len(parts) < 2:
        return [] # Retourne une liste vide si la ligne est mal formée

    label = parts[0]
    text = parts[1]

    # Vérifier le label
    if label != "TRUE" and label != "FAKE":
        return []

    # Nettoyer le texte : minuscules, enlève ponctuation (garde que lettres/espaces)
    text_cleaned = re.sub(r'[^a-zA-Z\s]', '', text.lower())

    # Tokenize (séparer en mots)
    words = text_cleaned.split()

    # Transformer en paires ( (label, word), 1 )
    results = []
    for word in words:
        # Appliquer notre filtre de stop words
        if word not in stop_words and len(word) > 3:
            composite_key = (label, word) # La clé est un tuple
            results.append((composite_key, 1))

    return results

# flatMap applique la fonction et "aplatit" les listes de listes
pairedRDD = baseRDD.flatMap(map_line_to_words)


# RDD 3 : reducedRDD (Le "Reducer")
# Grouper par la clé composite (label, word) et additionner les 1
countsRDD = pairedRDD.reduceByKey(lambda a, b: a + b)


# RDD 4 : sortedRDD (Trier les résultats)
# On trie par le 2ème élément du tuple (le compte), en ordre décroissant
sortedRDD = countsRDD.sortBy(lambda pair: pair[1], ascending=False)


# --- Action (Lancer le calcul) ---

# Récupérer les 100 premiers résultats
top_100_words = sortedRDD.take(100)

# Afficher les résultats
print("--- Top 100 des mots ---")
for (key, count) in top_100_words:
    print(f"{key[0]}\t{key[1]}\t{count}")

# Arrêter la session Spark
spark.stop()

--- Top 100 des mots ---
FAKE	trump	73933
TRUE	trump	42601
FAKE	about	32803
FAKE	their	30205
TRUE	reuters	28404
FAKE	people	25963
FAKE	president	25586
TRUE	president	25548
FAKE	been	22984
FAKE	more	22240
TRUE	which	20808
FAKE	just	20176
TRUE	been	19604
TRUE	after	19375
FAKE	there	18852
TRUE	state	18757
FAKE	clinton	18011
TRUE	government	17980
FAKE	obama	17813
TRUE	states	17639
FAKE	which	17527
TRUE	about	17449
FAKE	donald	17215
TRUE	more	17211
TRUE	their	16825
FAKE	after	16821
TRUE	house	16407
TRUE	also	15952
TRUE	united	15572
FAKE	because	15486
TRUE	republican	15292
FAKE	also	15242
TRUE	people	15117
TRUE	told	14244
FAKE	news	14126
FAKE	over	13921
TRUE	over	13854
FAKE	them	13746
FAKE	even	13717
FAKE	than	13716
TRUE	could	13705
FAKE	hillary	13565
FAKE	into	13390
FAKE	other	13216
FAKE	white	12778
FAKE	only	12776
TRUE	some	12754
FAKE	time	12728
TRUE	last	12614
FAKE	state	12525
TRUE	washington	12143
FAKE	being	12084
TRUE	trumps	11752
FAKE	some	11609
TRUE	other	11561
TRUE	election	11504
TRU